In [2]:
import advent
advent.scrape(2024, 21)
data = advent.get_lines(21)
print(data)

['208A', '540A', '685A', '879A', '826A']


In [3]:
from typing import NamedTuple, Iterator
from advent.maze import solve_maze, solve_maze_no_tqdm

class Node(NamedTuple):
    bottom: str # the main keypad
    middle: str # The radiated keypad
    top: str    # The -40 degrees keypad
    code: str   # Code that has been entered so far
    target_code: str # Target code that must be entered

def is_target(node):
    return node.code == node.target_code


bottom_adj = {
    '0': {'^': '2', '>': 'A'},
    '1': {'>': '2', '^': '4'},
    '2': {'v': '0', '<': '1', '>': '3', '^': '5'},
    '3': {'<': '2', '^': '6', 'v': 'A'},
    '4': {'v': '1', '>': '5', '^': '7'},
    '5': {'v': '2', '<': '4', '>': '6', '^': '8'},
    '6': {'v': '3', '<': '5', '^': '9'},
    '7': {'v': '4', '>': '8'},
    '8': {'v': '5', '<': '7', '>': '9'},
    '9': {'v': '6', '<': '8'},
    'A': {'<': '0', '^': '3'}
}

topmid_adj = {
    '<': {'>': 'v'},
    '^': {'v': 'v', '>': 'A'},
    'v': {'<': '<', '^': '^', '>': '>'},
    '>': {'<': 'v', '^': 'A'},
    'A': {'<': '^', 'v': '>'}
}

In [4]:
def adjacent(node: Node) -> Iterator[tuple[Node, int]]:
    # We pressed the wrong button somewhere: this node is a dead end
    if not node.target_code.startswith(node.code): return []
    # We can press either arrow or A
    # Press arrow: top moves. we only press the moves top has available
    for dir in topmid_adj[node.top]:
        yield Node(node.bottom, node.middle, topmid_adj[node.top][dir], node.code, node.target_code), 1
    # Press A: top stays the same. Now either top was arrow or A. first: top was arrow
    if node.top != 'A':
        # Middle will move according to what node.top was
        if node.top in topmid_adj[node.middle]:
            yield Node(node.bottom, topmid_adj[node.middle][node.top], node.top, node.code, node.target_code), 1
    # Press A, and top was also A. Middle will stay the same. Now either middle was arrow or A.
    elif node.middle != 'A':
        # bottom will move according to what node.middle was
        if node.middle in bottom_adj[node.bottom]:
            yield Node(bottom_adj[node.bottom][node.middle], node.middle, node.top, node.code, node.target_code), 1
    # Top and middle were both A. that means all robots will stay the same, and the code will be updated
    else:
        yield Node(node.bottom, node.middle, node.top, node.code + node.bottom, node.target_code), 1

start = Node('A', 'A', 'A', '', '029A')
l, _, _ = solve_maze_no_tqdm(start, is_target, adjacent)
assert l == 68
    

In [5]:
result = 0
for line in data:
    start = Node('A', 'A', 'A', '', line)
    l, _, _ = solve_maze_no_tqdm(start, is_target, adjacent)
    result += int(line[:3]) * l
print(result)

224326


In [6]:
# Part 2 :D
# This cell can be skipped, was a failed attempt because the search space increases exponentially with the number of robots
from functools import cache

class Node2(NamedTuple):
    bottom: str
    middles: tuple[str, ...]
    code: str
    target_code: str

@cache
def apply_press(middles: tuple[str, ...], button: str) -> None|tuple[tuple[str, ...], None|str]:
    # Returns None if this button may not be pressed
    if len(middles) == 0: return (), button
    top, rest = middles[0], middles[1:]
    if button != 'A':
        if button in topmid_adj[top]:
            return (topmid_adj[top][button],) + rest, None
        else:
            return None
    else:
        # top will stay the same. for the rest, use recursion
        subresult = apply_press(rest, top)
        if subresult is not None: return (top,) + subresult[0], subresult[1]
        else: return None

def adjacent2(node: Node2) -> Iterator[tuple[Node2, int]]:
    # We pressed the wrong button somewhere: this node is a dead end
    if not node.target_code.startswith(node.code): return []
    # We can press either arrow or A
    # Press arrow: top moves. we only press the moves top has available
    for buttonpress in ['<', 'v', '>', '^', 'A']:
        newmiddles = apply_press(node.middles, buttonpress)
        if newmiddles is None: continue # This press is not allowed
        newmiddles, bottombutton = newmiddles
        if bottombutton == 'A':
            yield Node2(node.bottom, node.middles, node.code + node.bottom, node.target_code), 1
        elif bottombutton is not None:
            if bottombutton in bottom_adj[node.bottom]:
                yield Node2(bottom_adj[node.bottom][bottombutton], newmiddles, node.code, node.target_code), 1
        else:
            yield Node2(node.bottom, newmiddles, node.code, node.target_code), 1
        
middle_keypads = 2  # 2 for part 1, 25 for part 2

result = 0
for line in data:
    start = Node2('A', ('A',) * middle_keypads, '', line)
    l, n, _ = solve_maze_no_tqdm(start, is_target, adjacent2)
    print(line, int(line[:3]), l)
    result += int(line[:3]) * l
print(result)

208A 208 70
540A 540 72
685A 685 68
879A 879 70
826A 826 76
224326


In [160]:
# Unfortunately this seems to get exponentially slower and is already pretty slow at just 7 keypads.
# So we have to think of something more clever...

# I realized that the transitions look the same one level higher. so e.g.
# <A always becomes x>>^A (where x is what presses the < button, ending with A)
# So we can encode what each layer needs as pairs, e.g. <A^ --> (<, A), (A, ^)

from collections import Counter

def required_for(pair: tuple[str, str], fullstr=False):
    if pair[0] == pair[1]: return 'A' if fullstr else [('A', 'A')]
    # WARNING! This only returns one possible path, specifically from A to V or < and ^to>
    s = 'A' + {
        ('v', 'A'): '^>A', ('v', '<'): '<A', ('v', '^'): '^A', ('v', '>'): '>A',
        ('<', 'v'): '>A',('<', 'A'): '>>^A', ('<', '^'): '>^A', ('<', '>'): '>>A', 
        ('^', 'v'): 'vA', ('^', '<'): 'v<A', ('^', 'A'): '>A', ('^', '>'): 'v>A',
        ('>', 'v'): '<A', ('>', '<'): '<<A', ('>', '^'): '<^A', ('>', 'A'): '^A',
        ('A', 'v'): '<vA', ('A', '<'): 'v<<A', ('A', '^'): '<A', ('A', '>'): 'vA',
    }[pair]
    if fullstr: return s[1:]
    return [(s[i], s[i+1]) for i in range(len(s) - 1)]

def counter_fullstr(line: str):
    pairs = [(line[i], line[i+1]) for i in range(len(line) - 1)]
    return 'A'+ ''.join([required_for(p, True) for p in pairs])

def counter_step(c: Counter) -> Counter:
    result = Counter()
    for pair, count in c.items():
        result.update(dict((p, count) for p in required_for(pair)))
    return result

def almost_full_sulotion(line: str, steps = 2):
    # example: line = 'A' + '<A^A>^^AvvvA' 
    pairs = [(line[i], line[i+1]) for i in range(len(line) - 1)]
    counter = Counter()
    counter.update(pairs)
    while steps > 0:
        #print(counter)
        steps -= 1
        counter = counter_step(counter)
    return counter.total(), counter

# just a proof of concept, ignoring the lowest keypad layer
line = 'A' + '<A^A>^^AvvvA'  # need A in front for starting position
assert almost_full_sulotion(line, steps=2)[0] == 68

# At this point I was kinda sick of this assignment
# But this is just kinda dumb... Apparently it only gives the correct answer if you put A<^^^A, not A^^^<A.
# Can't be bothered at this point to find out why...
lines_dict = {
    '208A': ['A<^AvA^^^Avvv>A', 'A^<AvA^^^Avvv>A', 'A<^AvA^^^A>vvvA', 'A^<AvA^^^A>vvvA'],
    '540A': ['A<^^A<A>vvA>A', 'A^^<A<A>vvA>A', 'A<^^A<Av>vA>A', 'A^^<A<Av>vA>A'],
    '685A': ['A^^A<^AvAvv>A', 'A^^A^<AvAvv>A', 'A^^A<^AvA>vvA', 'A^^A^<AvA>vvA'],
    '879A': ['A<^^^A<A>>AvvvA', 'A^^^<A<A>>AvvvA'],
    '826A': ['A<^^^AvvA^>AvvA', 'A^^^<AvvA^>AvvA', 'A<^^^AvvA>^AvvA', 'A^^^<AvvA>^AvvA']
}

def sorta_full_solution(line, steps=25):
    return min(almost_full_sulotion(l, steps=steps)[0] for l in lines_dict[line])


print(almost_full_sulotion(lines_dict['685A'][0], steps=2)[0])
print(almost_full_sulotion(lines_dict['879A'][0], steps=2)[0])
print(almost_full_sulotion(lines_dict['826A'][0], steps=2)[0])


assert almost_full_sulotion(lines_dict['208A'][0], steps=2)[0] == 70
assert almost_full_sulotion(lines_dict['540A'][0], steps=2)[0] == 72
assert almost_full_sulotion(lines_dict['685A'][0], steps=2)[0] == 68
assert almost_full_sulotion(lines_dict['879A'][0], steps=2)[0] == 70
assert almost_full_sulotion(lines_dict['826A'][0], steps=2)[0] == 76



68
70
76


In [161]:
print(208 * sorta_full_solution('208A', 2) + \
    540 * sorta_full_solution('540A', 2) + \
    685 * sorta_full_solution('685A', 2) + \
    879 * sorta_full_solution('879A', 2) + \
    826 * sorta_full_solution('826A', 2))

224326


In [163]:
result = 208 * sorta_full_solution('208A') + \
         540 * sorta_full_solution('540A') + \
         685 * sorta_full_solution('685A') + \
         879 * sorta_full_solution('879A') + \
         826 * sorta_full_solution('826A')
print(result)

279638326609472


In [ ]:
# What ended up solving it: in the 'required_for', I manually changed every path and ran it, to see what would give the lowest result lol
# Luckily I didn't need to test every combination of paths, just individually changing every path to see which one gives the lowest result worked.